In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import imgaug.augmenters as iaa
from PIL import Image
import os
from tqdm import tqdm
import numpy as np

# Set the path to your dataset
dataset_path = '/content/drive/MyDrive/Colab/dataset-augmented2'

# Create an augmentation sequence
augmentation_seq = iaa.Sequential([
    iaa.Fliplr(0.2),  # horizontally flip 50% of the images
    iaa.Flipud(0.4),  # vertically flip 20% of the images
    iaa.GaussianBlur(sigma=(0, 1.0)),  # apply gaussian blur with a sigma between 0 and 1.0
    iaa.AdditiveGaussianNoise(scale=(0, 0.1 * 255)),  # add gaussian noise
    iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25),  # apply elastic transformations
    iaa.Sometimes(0.5, iaa.GammaContrast((0.5, 2.0))),  # adjust gamma contrast with a 50% chance
])

# Set the desired number of images for each class (balance)
desired_num_images_per_class = 40

# Iterate over each class in your dataset
for class_folder in tqdm(os.listdir(dataset_path)):
    class_path = os.path.join(dataset_path, class_folder)

    # Get the current number of images in the class
    num_images_in_class = len(os.listdir(class_path))

    # Calculate the number of augmentations needed to reach the desired balance
    augmentations_needed = max(0, desired_num_images_per_class - num_images_in_class)

    print(num_images_in_class, augmentations_needed)

    # Iterate over each image in the class folder
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)

        # Load the image using PIL
        img = Image.open(image_path)

        # Convert PIL Image to numpy array
        img_array = np.array(img)

        # Apply augmentation
        for _ in range(augmentations_needed):
            # Check if the desired number of augmentations has been reached
            if len(os.listdir(class_path)) >= desired_num_images_per_class:
              break
            augmented_img_array = augmentation_seq.augment_image(img_array)

            # Save the augmented image in the same folder
            augmented_image_path = os.path.join(class_path, f"aug_{image_name}")
            Image.fromarray(augmented_img_array).save(augmented_image_path)

# Note: This code assumes that the number of original images in each class is less than or equal to the desired number of images per class.


  0%|          | 0/30 [00:00<?, ?it/s]

32 8


  3%|▎         | 1/30 [00:16<08:10, 16.92s/it]

21 19


  7%|▋         | 2/30 [01:02<15:39, 33.57s/it]

...

 93%|█████████▎| 28/30 [08:46<00:43, 21.86s/it]

23 17


 97%|█████████▋| 29/30 [08:56<00:18, 18.17s/it]

33 7


100%|██████████| 30/30 [09:06<00:00, 18.23s/it]


In [ ]:
import os
import pandas as pd
import numpy as np
import random
import shutil
from shutil import copyfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
import matplotlib.patches as mpatches

# Set the path to your dataset
dataset_path = '/content/drive/MyDrive/Colab/dataset-augmented2'
os.chdir(dataset_path)

# to list every directory name (label name)
directories_list = tf.io.gfile.listdir(dataset_path)

# get number of labels
len_labels = len(directories_list)
print(f"Total Class Labels = {len_labels}")

length_file_list = []; label_list = []

for item in directories_list:
    # get each label directory
    item_dir = os.path.join(dataset_path, item)
    # get list of images of each label
    item_files = os.listdir(item)
    # number of images per label
    len_per_label = len(os.listdir(item))

    length_file_list.append(len_per_label)
    label_list.append(item)

df_temp = pd.DataFrame({'Labels':label_list, 'Number of Images':length_file_list}).\
sort_values(by='Number of Images', ascending=False)
df_temp

Total Class Labels = 30


,Labels,Number of Images
0,cotton,40
1,almond,40
28,tea,40
27,vigna-radiati(Mung),40
26,wheat,40
25,rice,40
24,soyabean,40
23,sugarcane,40
22,tomato,40
21,pineapple,40
